In [1]:
from __future__ import print_function, division, absolute_import

from copy import copy
# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

from cobrame.core.component import (Constraint, ProcessedProtein, Complex,
                                    TranslatedGene, TranscribedGene, Metabolite, Ribosome,
                                    GenericComponent,GenerictRNA,RNAP)

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [ ]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
bsub_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')
bsub_ref_directory = join(flat_files.ecoli_files_dir, 'Bacillus_subtilis_subsp_subtilis_str_168.xml')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(bsub_directory)
bsub_ref = cobra.io.read_sbml_model(bsub_ref_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7f3c108a79e8>

In [ ]:
with open('./me_models/iBS.pickle', 'rb') as f:
    me = pickle.load(f)
    
with open('./me_models/iJL1678b.pickle', 'rb') as f:
    eco_me = pickle.load(f)

In [ ]:
GE_dict_m = gene_essentiality(bsub, model_type = 'm',  lim = 0.01, NP = 20)

In [ ]:
GE_dict_me = gene_essentiality(me, model_type = 'me',  lim = 0.01, NP = 20, initial_f=0)

In [ ]:
# Create one dict
GE_dict = {}
for key in GE_dict_me.keys():
    GE_dict[key] = {}
    try:
        GE_dict[key]['m'] = GE_dict_m[key]
    except:
        GE_dict[key]['m'] = None
    GE_dict[key]['me'] = GE_dict_me[key]
GE_dict

In [15]:
with open("./me_models/GE_dict.pickle", "wb") as outfile:
    pickle.dump(GE_dict, outfile)

In [10]:
r = get_reactions_of_met(me,'protein_BSU04560',0)

( formation_BSU04560-MONOMER 0.0 1000.0 ) 	 protein_BSU04560 --> BSU04560-MONOMER
( translation_BSU04560 0.0 1000.0 ) 	 10fthf_c + 0.00150854700854701*mu BSU01120-MONOMER + 4.27350427350427e-6*mu BSU01390-MONOMER + 4.27350427350427e-6*mu BSU15720-MONOMER + 4.27350427350427e-6*mu BSU15730-MONOMER + 0.00150854700854701*mu BSU16500-MONOMER + 4.27350427350427e-6*mu BSU16520-MONOMER + 8.54700854700855e-6*mu BSU16630-MONOMER + 4.27350427350427e-6*mu BSU28870-MONOMER + 4.27350427350427e-6*mu BSU35290-MONOMER + 0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_BSU04560 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 33 ala__L_c + 9 arg__L_c + 14 asn__L_c + 19 asp__L_c + 353.0 + 1.59089163237311*(mu + 0.3915)/mu atp_c + 4 cys__L_c + 4.27350427350427e-6*mu generic_RF + 0.00150854700854701*mu generic_Tuf + 20.0 generic_tRNA_AAA_lys__L_c + 9.0 generic_tRNA_AAC_asn__L_c + 5.0 generic_tRNA_AAG_lys__L_c + 5.0 generic_tRNA_AAU_asn__L_c + 8.0 gen

In [17]:
bsub.solution.f

0.11796638935761272

In [61]:
get_metabolites_from_pattern(me,'cl_c')

['shcl_c']

In [113]:
met = 'protein_b3035'
print(eco_me.metabolites.get_by_id(met).name)
print_reactions_of_met(eco_me,met,1)

In [6]:
met = 'fmn_c'
print(me.metabolites.get_by_id(met).name)
print_reactions_of_met(me,met,-1)

FMN C17H19N4O9P


NameError: name 'print_reactions_of_met' is not defined

In [57]:
for met in bsub.metabolites:
    if 'complex' in met.name:
        print(met.id, met.name)

In [15]:
eco_me.metabolites.get_by_id('GRXB-MONOMER_mod_Oxidized').formula_weight

24349.923540000003

In [5]:
eco_me.metabolites.get_by_id('heme_c').formula

KeyError: 'heme_c'

In [15]:
bsub.metabolites.get_by_id('ACP_c').formula

'HX'

In [42]:
me.reactions.DNA_replication.metabolites

{<Metabolite datp_c at 0x7f6228fa36a0>: -0.246051501452,
 <Metabolite dctp_c at 0x7f6228fa35f8>: -0.253948498548,
 <Metabolite dgtp_c at 0x7f6228fa31d0>: -0.253948498548,
 <Metabolite dttp_c at 0x7f6228fa33c8>: -0.246051501452,
 <Metabolite ppi_c at 0x7f622fe6b828>: 1,
 <Constraint DNA_biomass at 0x7f622d17b6a0>: 0.3079428373534753}

In [37]:
r = get_reactions_of_met(bsub,'ac_c',0)

( ACS 0.0 999999.0 ) 	 ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
( ADPDA -999999.0 999999.0 ) 	 26dapime_c + h2o_c <=> 26dap_LL_c + ac_c
( ALDD2x 0.0 999999.0 ) 	 acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
( ACODA_1 0.0 999999.0 ) 	 acorn_c + h2o_c --> ac_c + orn__L_c
( CYSS_2 0.0 999999.0 ) 	 acser_c + h2s_c --> ac_c + cys__L_c
( AHSERL4 -999999.0 999999.0 ) 	 acser_c + trdrd_c + tsul_c <=> ac_c + cys__L_c + h_c + so3_c + trdox_c
( AGDC 0.0 999999.0 ) 	 acgam6p_c + h2o_c --> ac_c + gam6p_c
( AHSERL3 -999999.0 999999.0 ) 	 acser_c + seln_c <=> ac_c + scys__L_c
( SLCYSS 0.0 999999.0 ) 	 acser_c + tsul_c --> ac_c + scys__L_c
( ACKr -999999.0 999999.0 ) 	 ac_c + atp_c <=> actp_c + adp_c
( ACt2r -999999.0 999999.0 ) 	 ac_e + h_e <=> ac_c + h_c


In [31]:
i=0
for rxn in me.reactions:
    rule = rxn.gene_reaction_rule
    if 'dummy' in rxn.id:
        i = i + 1

print(i)

182


In [7]:
eco_me.reactions.ATPM.gene_reaction_rule

''

In [25]:
rxn_id = 'PYRt2rpp'
eco.reactions.get_by_id(rxn_id).reaction
eco_me.reactions.SELGTHR2_FWD_CPLX_dummy.gene_reaction_rule

''